In [1]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

from src.lab08.langchain_tool import ai_message

In [2]:
load_dotenv()  # .env 파일의 api_key 정보를 환경 변수로 로딩.

True

# 한 번에 출력하기

## 언어 모델만 있는 경우 한 번에 출력하기

In [3]:
model = ChatOpenAI(model='gpt-4o-mini')

In [4]:
parser = StrOutputParser()

In [5]:
chain = model | parser

In [6]:
messages = [HumanMessage('한강 작가의 채식주의자를 1000자 이내로 요약해줘.')]

In [7]:
ai_message = chain.invoke(input=messages)

In [8]:
print(ai_message)

한강의 소설 『채식주의자』는 현대 사회에서 개인의 선택과 정체성, 가족과 사회의 갈등을 탐구하는 작품입니다. 이야기는 주인공 영혜가 갑자기 채식주의자가 되면서 시작됩니다. 그녀는 고기를 거부하고 완전히 채식으로 전환하는데, 이는 가족과 주변 사람들에게 큰 충격을 안깁니다.

소설은 세 개의 파트로 나뉘어 있으며, 각 파트는 다른 시점에서 영혜의 삶을 다룹니다. 첫 번째 파트는 남편의 시점에서 영혜의 변화를 바라보는 이야기로, 그녀의 선택이 가정에 미치는 영향을 보여줍니다. 두 번째 파트에서는 영혜의 형부가 등장해 그녀에게 매력을 느끼게 되며, 이로 인해 발생하는 비극적인 사건이 그려집니다. 마지막 파트는 영혜의 언니의 시점으로, 그녀의 복잡한 감정을 통해 가족의 붕괴와 영혜의 고통을 조명합니다.

채식주의가 단순한 식습관의 변화가 아니라 존재에 대한 깊은 질문으로 이어지면서, 작가는 인간의 삶에 대한 심오한 성찰을 담고 있습니다. 이 작품은 변증법적 요소를 통해 인간의 본성과 욕망, 그리고 그것이 사회와 어떻게 얽히는지를 탐구하여 독자에게 강렬한 여운을 남깁니다.


## 도구를 설정한 경우 한 번에 출력하기

# 스트리밍 방식으로 출력하기